# 단어
Retrospective Study

## Background
제목으로 사용했던 여러가지 단어를 전부 추출한 뒤   
조회수가 높은 단어를 찾는다.

In [5]:
api_key = [API_KEY]
channel_id = "UCH98-54LGg386084raqob6g"

In [6]:
from googleapiclient.discovery import build
youtube = build("youtube", "v3", developerKey=api_key)

In [7]:
def get_channel_videos(channel_id):
    res = youtube.channels().list(id=channel_id,
                                 part="contentDetails").execute()
    playlist_id = res["items"][0]["contentDetails"]["relatedPlaylists"]["uploads"]
    
    videos = []
    next_page_token = None
    
    while 1:
        res = youtube.playlistItems().list(playlistId=playlist_id,
                                          part="snippet",
                                          maxResults=50,
                                          pageToken=next_page_token).execute()
        videos += res["items"]
        next_page_token = res.get("nextPageToken")
        
        if next_page_token is None:
            break
    
    return videos

videos = get_channel_videos(channel_id)

In [8]:
video_ids = list(map(lambda x:x['snippet']['resourceId']['videoId'], videos))

def get_videos_stats(video_ids):
    stats = []
    for i in range(0, len(video_ids), 50):
        res = youtube.videos().list(id=','.join(video_ids[i:i+50]),
                                   part='statistics').execute()
        stats += res['items']
    return stats

stats = get_videos_stats(video_ids)

In [29]:
title = []
for video in videos:
    title.append(video['snippet']['title'])

In [30]:
viewCount = []
for stat in stats:
    viewCount.append(stat['statistics']['viewCount'])

In [31]:
import pandas as pd
df = pd.DataFrame(zip(title, viewCount), columns=['title','viewCount'])
df['viewCount'] = df['viewCount'].astype(int)

## Analysis

### morphs

In [108]:
from konlpy.tag import Okt
okt = Okt()

sentence = []
for t in df['title']:
    sentence.append(okt.morphs(t))

word = sum(sentence,[])

In [113]:
from collections import Counter
count = Counter(word).most_common()[:200]

In [115]:
dict = {}
for i in range(len(count)):
    try:
        dict[count[i][0]] = round(df[df['title'].str.contains(count[i][0]) == True]['viewCount'].mean())
    except:
        pass

In [119]:
sorted_dict = sorted(dict.items(), key = lambda item: item[1], reverse = True)
print(sorted_dict[:100])

[('무상증자', 10911), ('박스', 9685), ('곤', 9062), ('진양', 9009), ('나노', 8940), ('NDA', 8564), ('승인', 7900), ('회장', 7765), ('FDA', 7169), ('발표', 6734), ('신약', 6489), ('허가', 6477), ('가능성', 6465), ('증', 6363), ('옵티머스', 6297), ('에이치엘비', 6160), ('공시', 6111), ('어떻게', 6110), ('f', 6036), ('공매도', 5847), ('개발', 5802), ('!', 5761), ('했습니다', 5743), ('병용', 5733), ('유상증자', 5723), ('나', 5656), ('사례', 5639), ('미국', 5594), ('약', 5491), ('이뮤노믹', 5489), ('에', 5478), ('주주', 5473), ('제약', 5449), ('닙', 5443), ('세라', 5423), ('치료', 5367), ('리보', 5344), ('!!', 5319), ('일까', 5316), ('백신', 5224), ('코', 5219), ("'", 5140), ('부터', 5122), ('아파티닙', 5075), ('인수', 5075), ('하면', 5074), ('생명과학', 5038), ('것', 5011), ('시작', 4970), ('매도', 4967), ('이', 4955), ('후', 4908), ('된', 4873), ('임', 4814), ('백', 4770), ('항암제', 4755), ('OS', 4709), ('있다', 4634), ('결과', 4596), ('사이언스', 4592), ('까지', 4589), ('기사', 4536), ('항', 4515), ('제', 4480), ('그', 4446), ('3', 4439), ('기업', 4395), ('에서', 4369), ('고', 4362), ('서', 4356), ('임상시험', 4351),

### nouns

In [120]:
from konlpy.tag import Okt
okt = Okt()

sentence = []
for t in df['title']:
    sentence.append(okt.nouns(t))

word = sum(sentence,[])

In [121]:
from collections import Counter
count = Counter(word).most_common()[:200]

In [122]:
dict = {}
for i in range(len(count)):
    try:
        dict[count[i][0]] = round(df[df['title'].str.contains(count[i][0]) == True]['viewCount'].mean())
    except:
        pass

In [123]:
sorted_dict = sorted(dict.items(), key = lambda item: item[1], reverse = True)
print(sorted_dict[:100])

[('무상증자', 10911), ('선위', 10134), ('박스', 9685), ('라이브', 9650), ('방송', 9326), ('곤', 9062), ('진양', 9009), ('나노', 8940), ('실시간', 8176), ('승인', 7900), ('회장', 7765), ('폴첸', 7559), ('추가', 7279), ('권리', 7120), ('박사', 6985), ('잔고', 6914), ('발표', 6734), ('관해', 6647), ('완전', 6625), ('신약', 6489), ('허가', 6477), ('가능성', 6465), ('증', 6363), ('유튜브', 6333), ('옵티머스', 6297), ('계약', 6296), ('신청', 6164), ('에이치엘비', 6160), ('엘', 6112), ('공시', 6111), ('에이', 6086), ('로열티', 6077), ('사용', 6065), ('합병', 5907), ('공매도', 5847), ('증가', 5833), ('개발', 5802), ('병용', 5733), ('유상증자', 5723), ('시간', 5663), ('나', 5656), ('코로나', 5652), ('전환사채', 5646), ('사례', 5639), ('미국', 5594), ('이뮤노믹', 5489), ('주주', 5473), ('제약', 5449), ('닙', 5443), ('세라', 5423), ('치료', 5367), ('리보', 5344), ('백신', 5224), ('코', 5219), ('보고', 5106), ('전환', 5105), ('아파티닙', 5075), ('인수', 5075), ('필', 5056), ('생명과학', 5038), ('것', 5011), ('글로벌', 4973), ('시작', 4970), ('매도', 4967), ('이', 4955), ('후', 4908), ('임', 4814), ('건', 4814), ('외국인', 4792), ('항암제', 4755), ('

## Results
'NDA' 조회수 평균: 8564회  
'FDA' 조회수 평균: 7169회  
'발표' 조회수 평균: 6734회  
'가능성' 조회수 평균: 6464회  
'!' 조회수 평균: 5761회  
'사례' 조회수 평균: 5639회

## Conclusion
추가 연구에서 각 단어의 높은 조회수가 통계적으로 유의미한지 t-test한다.